In [1]:
import os, os.path, qgrid, math
import pandas as pd

from framenet.builder         import build
from framenet.ecg.generation  import unstack_all
from framenet.util            import (aget, iget, compose, groupby, curry, 
                                      groupwise, remove, reduceby, flatmap, merge)
from framenet.data.annotation import annoset_for, to_list, Pattern, Graph
# from collections             import defaultdict
from pprint                   import pprint, pformat
from collections              import namedtuple


def lus_for(frame_name):
    """LUs for `frame` as a pd.DataFrame.
    """
    cs    = ('annotationSet.ID', 'annotationSet.LU')
    lu    = aget('ID', 'lu')
    fn, _ = build()
    frame = fn.get_frame(frame_name)
    return pd.DataFrame([dict(zip(cs, lu(ann))) for ann in frame.annotations])
    

def core_FE_for(frame_name):
    fn, _ = build()
    frame = fn.get_frame(frame_name)
    return set(int(e.ID) for e in frame.elements if e.coreType == 'Core')
    
    
def get_frame_df(frame_name):
    """Get pd.DataFrame for `frame_name`, normalize by adding Target LUs using the above.
    """
    lu_df       = get_lu_df()
    aset_ids    = annoset_for(frame_name)
    core_fe_ids = core_FE_for(frame_name)
    selected    = lu_df.loc[lu_df['annotationSet.ID'].isin(aset_ids)]
    merged      = pd.merge(selected, lus_for(frame_name), how='outer', on='annotationSet.ID')
    merged.loc[
        merged['label.name'] == 'Target', 
        'label.name'
    ] = merged['annotationSet.LU']
#     merged.loc[
#         merged['label.start'].isnull(),
#         'label.start'
#     ] = 9999
    merged.loc[
        merged['label.feID'].isin(core_fe_ids),
        'label.coreFE'
    ] = True
    return merged


def get_lu_df(base='.'):
    """Return LU records as a pd.DataFrame.
    """
    LU_PICKLE      = 'lu.pkl'
    lu_pickle_path = os.path.join(base, LU_PICKLE)
    if os.access(lu_pickle_path, os.R_OK):
        # Read lu_df back in
        lu_df = pd.read_pickle(lu_pickle_path)
        return lu_df
    else:
        # Save to a file in the current directory
        lu_df = pd.DataFrame(list(unstack_all(lu_sents())))
        lu_df.to_pickle(lu_pickle_path)
        return lu_df


def test_get_frame_df(frame_name='Cause_motion', count=10788):
    cm_df = get_frame_df(frame_name)
    assert len(cm_df) == 10788, 'Test falied: %d' % len(cm_df)
    
%time test_get_frame_df()

CPU times: user 4.3 s, sys: 2.24 s, total: 6.54 s
Wall time: 7.62 s


In [4]:
lu_df = get_lu_df()

In [7]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///lu.db')
lu_df.to_sql('lu', engine)

In [25]:
cm_df = get_frame_df('Cause_motion', '.')
cm_df.info()

len(cm_df['annotationSet.ID'].unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10788 entries, 0 to 10787
Data columns (total 18 columns):
annotationSet.ID        10788 non-null int64
annotationSet.status    10788 non-null object
label.end               8209 non-null float64
label.feID              2655 non-null float64
label.itype             250 non-null object
label.name              8459 non-null object
label.start             8209 non-null float64
layer.name              10788 non-null object
layer.rank              10788 non-null int64
sentence.ID             10788 non-null int64
sentence.aPos           10788 non-null int64
sentence.corpID         1231 non-null float64
sentence.docID          1231 non-null float64
sentence.paragNo        1231 non-null float64
sentence.sentNo         10788 non-null int64
text.contents           10788 non-null object
annotationSet.LU        10788 non-null object
label.coreFE            2556 non-null object
dtypes: float64(6), int64(5), object(7)
memory usage: 1.6+ MB


823

In [3]:
qgrid.show_grid(cm_df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100})

In [4]:
# Ad-hoc function. TODO: make into a general one.

itypes = {k: i for i, k in enumerate(('CNI', 'INI', 'DNI'))}

def isnan(x):
    try:
        return math.isnan(x)
    except TypeError:
        return False
    

def make_groups(records):
    """Group by sentence.ID and label.start, hierarchically; return jsut the groups.
    """
#     print('>> make_groups...')
    
    def default_start(_, d):
#         print('>> default_start called!')
        k1, k2 = iget('label.itype', 'label.feID', default=100)(d)
        return 9999 + itypes.get(k1, 100) + 10 * k2
    
    def label_start(d):
        s = d.get('label.start')
        return default_start(None, d) if isnan(s) else s
        
    sentence_id                   = iget('sentence.ID') 
#     label_start                   = iget('label.start', default=default_start)
    grouped_by_sentence           = groupby(sentence_id, records)
    grouped_by_sentence_and_start = ((k, groupby(label_start, g))
                                     for k, g in grouped_by_sentence)
    return to_list(grouped_by_sentence_and_start, levels=2, with_keys=False)


fields = iget('layer.name', 'label.name', 'label.itype', 'label.coreFE', default=None)

def pivot(m, fields=fields):
    a, b, c, d_ = fields(m)
    d = False if isnan(d_) else True
#     if not isnan(c):
#         print('a:', a, 'b:', b, 'c:', c)
#     assert not isnan(c) and b, 'a: %s, b: %s, c: %s' % (a, b, c)
    if not isnan(c):
        return [(a, b), ('GF', c), ('PT', None)] + ([('core', d)] if d else [])
    elif not isnan(a) and not isnan(b):
        return [(a, b)] + ([('core', d)] if d else [])
    else:
        return []

layer_names = ('GF', 'PT', 'FE', 'Target', 'CNI', 'DNI', 'INI')

def to_layers(dss, layer_names=layer_names):
    "Return layers in a seq of a seq of dictionaries (i.e., a single group)."
    def p(ds):
        return merge(dict(), 
                     [i for d in ds for i in pivot(d) 
                      if d['layer.name'] in layer_names and d['layer.rank'] == 1])
    try:
        ls = [p(ds) for ds in dss if p(ds)]
        #     pprint(ls)
        return ls
    except ValueError:
        pprint('Problem with group')
        pprint(dss)
        raise

In [5]:
gs = make_groups(cm_df.to_dict(orient='records'))

In [6]:
# pprint([layers(g) for g in gs][:10])
gs[7], to_layers(gs[7])
# ll = [to_layers(g) for g in gs]
# len(gs[0])
# pprint(gs[1])

([[{'annotationSet.ID': 642924,
    'annotationSet.LU': 'catapult.v',
    'annotationSet.status': 'MANUAL',
    'label.coreFE': True,
    'label.end': 1.0,
    'label.feID': 229.0,
    'label.itype': nan,
    'label.name': 'Theme',
    'label.start': 0.0,
    'layer.name': 'FE',
    'layer.rank': 1,
    'sentence.ID': 580537,
    'sentence.aPos': 64198570,
    'sentence.corpID': nan,
    'sentence.docID': nan,
    'sentence.paragNo': nan,
    'sentence.sentNo': 0,
    'text.contents': 'He swerved and avoided the worst of the impact , but was catapulted on to the bonnet of the car and then on to the pavement .'},
   {'annotationSet.ID': 642924,
    'annotationSet.LU': 'catapult.v',
    'annotationSet.status': 'MANUAL',
    'label.coreFE': nan,
    'label.end': 1.0,
    'label.feID': nan,
    'label.itype': nan,
    'label.name': 'Ext',
    'label.start': 0.0,
    'layer.name': 'GF',
    'layer.rank': 1,
    'sentence.ID': 580537,
    'sentence.aPos': 64198570,
    'sentence.corpID': nan

## Patterns

In [8]:
# Ext T Obj Dep+

gf_tgt = compose(iget('GF', 'Target', default=None))

def gf_or_target(group):
    "Turn a group into a pattern."
    f, s = gf_tgt(group) 
    return f or s


# Tests
g1 = [{'FE': 'Agent', 'GF': 'Ext', 'PT': 'NP'},
      {'Target': 'catapult.v'},
      {'FE': 'Theme', 'GF': 'Obj', 'PT': 'NP'},
      {'FE': 'Path',  'GF': 'Dep', 'PT': 'PP'},
      {'FE': 'Goal',  'GF': 'Dep', 'PT': 'PP'}]    

assert list(map(gf_or_target, g1)) == ['Ext', 'catapult.v', 'Obj', 'Dep', 'Dep']


g0 = to_layers(gs[0])

assert list(map(gf_or_target, g0)) == ['Ext', 'catapult.v', 'Obj', 'Dep', 'Dep']

    
def get_matcher(vertices):
    ps = groupwise(2)(vertices)
    def edge(ps):
        a, b = ps
        if b == '+':
            return a, [a]
        else:
            return a, [b]
    
    es = list(map(edge, ps))
#     print(es)
    return Pattern(Graph(vertices=vertices, edges=es))


matcher = get_matcher(['Ext', '_', 'Obj', 'Dep', '+'])

@curry
def match(matcher, group):
#     pprint(group)
    return matcher.match(map(gf_or_target, group))

assert match(matcher, g0)
assert match(matcher, g1)

g3 = [{'FE': 'Theme'},
      {'FE': 'Source'},
      {'FE': 'Agent'},
      {'GF': 'Ext', 'PT': 'NP'},
      {'GF': 'Dep', 'PT': 'PP'}],

## $Pattern_1$: `Ext _ Obj Dep+`

In [9]:
vertices1 = ['Ext', '_', 'Obj', 'Dep', '+']
pattern1  = get_matcher(vertices1)
matching1 = [g for g in gs if match(pattern1, to_layers(g))] 
count_matching1 = len(matching1)

print('For %s: %d, %.2f%%' % (vertices1, count_matching1, 100.0 * count_matching1 / len(gs)))

For ['Ext', '_', 'Obj', 'Dep', '+']: 453, 55.18%


In [10]:
Node   = namedtuple('Node', 'id FE GF core')
Link   = namedtuple('Link', 'source target')

fe_gf  = iget('FE', 'GF', 'core', default=None)

def to_node(group):
    "Make a single node from a non-empty `group`."
#     pprint(group)
    ls     = to_layers(group)
#     pprint(ls)
    fe_gfs = [fe_gf(l) for l in ls if fe_gf(l)[0]]
#     pprint(fe_gfs)
    try:
        return [Node('%d:%s' % (i, fe) 
                     , str(fe)
                     , str(gf)
                     , core
#                      , group[0][0]['text.contents']
                    ) 
            for i, (fe, gf, core) in enumerate(fe_gfs)]
    except ValueError:
        pprint(fe_gfs)
#         pprint(group)
        pprint(ls)
        raise


def cols(link):
    return ['%s_%s' % (st, k) 
            for st, n in zip(('source', 'target'), link)
            for k in n._asdict().keys()]

assert cols(Link(Node('id', 'fe', 'gf', True), 
                 Node('id2', 'fe2', 'gf2', False))) == [
    'source_id', 'source_FE', 'source_GF', 'source_core', #'source_text',
    'target_id', 'target_FE', 'target_GF', 'target_core'#, 'target_text'
]


def write_csv(fname, groups, noncore=False, base='.'):
    
    def to_csv(link_and_count):
        link, count = link_and_count
        return (link.source.id, link.source.FE, link.source.GF, link.source.core, #link.source.text,
                link.target.id, link.target.FE, link.target.GF, link.target.core, #link.target.text,
                count)

    def links(nodes):
#         pprint(nodes)
        if noncore or all(n.core for n in nodes):
            return [Link(*pair) for pair in groupwise(2, nodes)]
        else:
            return []

    path   = os.path.join(base, '%s.csv' % fname)

    with open(path, 'w+') as sout:
        ident           = lambda x: x
        count           = lambda ys, _: ys + 1
        nodes           = map(to_node, groups)
        link_and_counts = list(reduceby(ident, 0, count, flatmap(links, nodes)))
#         pprint(link_and_counts)
        for i, lc in enumerate(link_and_counts):
            if i == 0: # write header
                print(','.join(cols(lc[0]) + ['count']), file=sout)
            print(','.join(str(lc) for lc in to_csv(lc)), file=sout)
    print('Written %d records.' % i)

In [11]:
write_csv('pattern_1_core', matching1)

Written 22 records.


## $Pattern_2$: `Ext _ Dep+`

In [12]:
vertices2 = ['Ext', '_', 'Dep']
edges2    = (
    ('Ext', ['_']),
    ('_', ['Dep']),
    ('Dep', ['Dep', None, 'INI', 'CNI', 'DNI']),
    ('CNI', ['INI', 'CNI', 'DNI', None]),
    ('DNI', ['INI', 'CNI', 'DNI', None]),
    ('INI', ['INI', 'CNI', 'DNI', None]),
    (None, ['INI', 'CNI', 'DNI', None]),
)

# p = Ext & _ & OneOrMore(Dep.fe.Path | Dep.fe.Goal | Dep.fe.Source)
# Annotations.selectBy(p)

pattern2        = Pattern(Graph(vertices2, edges2))
matching2       = [g for g in gs if match(pattern2, to_layers(g))] 
count_matching2 = len(matching2)

print('For %s: %d, %.2f%%' % (vertices2, count_matching2, 100.0 * count_matching2 / len(gs)))

For ['Ext', '_', 'Dep']: 167, 20.34%


In [13]:
# [to_node(g) for g in gs]

In [14]:
write_csv('pattern_2_core', matching2, noncore=False)

Written 39 records.


In [16]:
m12 = matching1 + matching2


ns  = [to_node(g) for g in gs if g not in m12]
# ns[:2]

In [19]:
from framenet.data.annotation import to_layers, make_groups, get_frame_df, to_node, to_layers


cfm_df = get_frame_df('Cause_fluidic_motion', '.')
cfm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894 entries, 0 to 1893
Data columns (total 18 columns):
annotationSet.ID        1894 non-null int64
annotationSet.status    1894 non-null object
label.end               1406 non-null float64
label.feID              483 non-null float64
label.itype             68 non-null object
label.name              1474 non-null object
label.start             1406 non-null float64
layer.name              1894 non-null object
layer.rank              1894 non-null int64
sentence.ID             1894 non-null int64
sentence.aPos           1894 non-null int64
sentence.corpID         245 non-null float64
sentence.docID          245 non-null float64
sentence.paragNo        245 non-null float64
sentence.sentNo         1894 non-null int64
text.contents           1894 non-null object
annotationSet.LU        1894 non-null object
label.coreFE            442 non-null object
dtypes: float64(6), int64(5), object(7)
memory usage: 281.1+ KB


In [22]:
len(cfm_df['annotationSet.ID'].unique())

143

In [14]:
cfm_groups = make_groups(cfm_df.to_dict(orient='records'))

ns = [to_node(g) for g in cfm_groups]

In [15]:
from IPython.display import HTML
from collections     import Counter
from framenet.util   import iget

def rec(ns):
    return tuple('%s: %s' % (n.GF, n.FE) for n in ns)

gf_fe_tgt    = iget('GF', 'FE', 'Target', default='')
gf_fe_pt_tgt = iget('GF', 'FE', 'PT', 'Target', default='')

def tag(elem):
    def e(t, **kwargs): 
        if kwargs:
            avs = ' '.join('%s="%s"' % (k, v) for k, v in kwargs.items())
            return '<{0} {2}>{1}</{0}>'.format(elem, str(t), avs)
        else:
            return '<{0}>{1}</{0}>'.format(elem, str(t))
    return e

table, tr, td, th, b, ul, li = map(tag, 'table tr td th b ul li'.split())

def gf_or_(group):
    "Turn a group into a pattern."
    gf, fe, target = gf_fe_tgt(group) 
    return '%s: %s' % (b(gf), fe) if not target else 'v'

def gf_pt_or_(group):
    "Turn a group into a pattern."
    print(group)
    gf, fe, pt, target = gf_fe_pt_tgt(group) 
    return '%s: %s (%s)' % (b(gf), fe, pt) if not target else 'v'

cs = Counter([rec(n) for n in ns]).most_common()

align_r = 'text-align: right;'

def t(cs):
    "Make a table from counts."
    rows = [''.join((td(i + 1, style=style), 
                     td(c, style=style), 
                     td(' \u2192 '.join(ss))))
            for i, (ss, c) in enumerate(cs)
            if c > 4]
#     pprint (rows)
    header = tr(''.join(map(th, ('', 'freq.', 'Pattern'))))
#     pprint(header)
    return table(header + '\n'.join(map(tr, rows)))


def t2(pattern_and_ss):
    "Table from (`pattern`, `sentence list`) pairs."
    rows = ['\n'.join((td(i + 1,   style=align_r), 
                       td(len(ss), style=align_r), 
                       td(' \u2192 '.join(pattern), style='vertical-align: top; white-space: nowrap;'),
                       td(ul(''.join(li(s) for s in ss)), style='text-align: left;')))
            for i, (pattern, ss) in enumerate(pattern_and_ss)]
    header = tr(''.join(map(th, ('', 'freq.', 'Pattern', 'Sentences'))))
    return table(header + '\n'.join(map(tr, rows)))

    
def text(group):
    return group[0][0]['text.contents']

##  Patterns <u>not</u> matching  $Pattern_1$ and  $Pattern_2$ above

In [16]:
gts = [(tuple(gf_pt_or_(l) for l in to_layers(g)), text(g)) for g in cfm_groups]
gts[:2]

{'FE': 'Agent', 'core': True, 'GF': 'Ext', 'PT': 'NP'}
{'Target': 'splatter.v'}
{'FE': 'Fluid', 'core': True, 'GF': 'Obj', 'PT': 'NP'}
{'FE': 'Goal', 'core': True, 'GF': 'Dep', 'PT': 'PP'}
{'FE': 'Agent', 'core': True, 'GF': 'Ext', 'PT': 'NP'}
{'Target': 'splatter.v'}
{'FE': 'Fluid', 'core': True, 'GF': 'Obj', 'PT': 'NP'}
{'FE': 'Goal', 'core': True, 'GF': 'Dep', 'PT': 'PP'}
{'FE': 'Fluid', 'core': True, 'GF': 'Ext', 'PT': 'NP'}
{'Target': 'splatter.v'}
{'FE': 'Goal', 'core': True, 'GF': 'Dep', 'PT': 'PP'}
{'FE': 'Agent', 'GF': 'CNI', 'PT': None, 'core': True}
{'FE': 'Fluid', 'core': True, 'GF': 'Ext', 'PT': 'NP'}
{'Target': 'splatter.v'}
{'FE': 'Goal', 'core': True, 'GF': 'Obj', 'PT': 'NP'}
{'FE': 'Agent', 'GF': 'CNI', 'PT': None, 'core': True}
{'FE': 'Agent', 'core': True, 'GF': 'Ext', 'PT': 'NP'}
{'Target': 'splatter.v'}
{'FE': 'Fluid', 'core': True, 'GF': 'Obj', 'PT': 'NP'}
{'FE': 'Path', 'core': True, 'GF': 'Dep', 'PT': 'PP'}
{'FE': 'Goal', 'GF': 'DNI', 'PT': None, 'core': True}
{

[(('<b>Ext</b>: Agent (NP)',
   'v',
   '<b>Obj</b>: Fluid (NP)',
   '<b>Dep</b>: Goal (PP)'),
  'Brandon then told one of the other group : ` Order your food and leave if you do n\'t want trouble , or I \'ll splatter your face across the road . "'),
 (('<b>Ext</b>: Agent (NP)',
   'v',
   '<b>Obj</b>: Fluid (NP)',
   '<b>Dep</b>: Goal (PP)'),
  '` Bastard ! " cried Bernard , belabouring Jed with his fists , splattering cheap red wine over books and walls .')]

In [27]:
from collections import defaultdict

dd = defaultdict(list)
for i, (k, v) in enumerate(gts): dd[k].append(v)

print('total:', i)

p_ss = [(p, ss) 
        for p, ss in sorted(list(dd.items()), key=lambda p: len(p[1]), reverse=True)
       # if len(ss) > 4
       ]


HTML(t2(p_ss))

total: 142


In [90]:
HTML(t(cs))

,freq.,Pattern
1,29,Ext: Agent → Dep: Goal → Obj: Theme
2,13,Ext: Agent → Dep: Source → Obj: Theme
3,13,Obj: Theme → Dep: Goal → CNI: Agent
4,8,Dep: Goal → Obj: Theme → CNI: Agent
5,8,Ext: Agent → Dep: Path → Obj: Theme
6,7,Ext: Agent → Obj: Theme → INI: Goal
7,7,Obj: Theme → Dep: Path → CNI: Agent
8,6,Obj: Theme → Obj: Theme → Ext: Agent → Dep: Goal
9,5,Obj: Theme → Dep: Source → CNI: Agent
10,5,Ext: Agent → Ext: Agent → Obj: Theme → Dep: Path


In [55]:
cs2 = Counter(gts)
# cs2.most_common()

HTML(t(cs2.most_common()))

,freq.,Pattern
1,29,Ext: Agent → v → Dep: Goal → Obj: Theme
2,13,v → Obj: Theme → Dep: Goal → CNI: Agent
3,13,Ext: Agent → v → Dep: Source → Obj: Theme
4,8,Ext: Agent → v → Dep: Path → Obj: Theme
5,8,v → Dep: Goal → Obj: Theme → CNI: Agent
6,7,Ext: Agent → v → Obj: Theme → INI: Goal
7,7,v → Obj: Theme → Dep: Path → CNI: Agent
8,6,Obj: Theme → Obj: Theme → Ext: Agent → v → Dep: Goal
9,5,v → Obj: Theme → Dep: Source → CNI: Agent
10,5,Ext: Agent → Ext: Agent → v → Obj: Theme → Dep: Path


# To do:

- Get sentences for these
- Finish pattern recognizer! 
- distribution of annotations over frames and position in the hierarchy